In [10]:
from datetime import datetime, timedelta
from typing import List, Optional, Dict, Tuple

class Task:
    def __init__(self, amount: int, amount_asked: int, product: str, machine: int, start_date: datetime, duration: timedelta):
        self.amount = amount
        self.amount_asked = amount_asked
        self.product = product
        self.machine = machine
        self.start_date = start_date
        self.duration = duration
        self.end_date = start_date + duration

    def __repr__(self):
        return (f"Task(Product: {self.product}, Amount Asked: {self.amount_asked}, "
                f"Amount Able: {self.amount}, Machine: {self.machine}, "
                f"Start: {self.start_date}, End: {self.end_date})")

class GlassMaker:
    def __init__(self, num_machines: int, setup_times: Dict[Tuple[str, str], timedelta]):
        self.schedules: List[List[Task]] = [[] for _ in range(num_machines)]
        self.setup_times = setup_times
        self.num_machines = num_machines

    def can_insert(self, machine_schedule: List[Task], new_task: Task) -> bool:
        for task in machine_schedule:
            if not (new_task.end_date <= task.start_date or new_task.start_date >= task.end_date):
                return False
        return True

    def calculate_setup_time(self, machine_schedule: List[Task], new_task: Task) -> timedelta:
        if not machine_schedule:
            return timedelta(0)
        last_task = machine_schedule[-1]
        if last_task.product != new_task.product:
            return self.setup_times.get((last_task.product, new_task.product), timedelta(0))
        return timedelta(0)

    def insert_task(self, amount_asked: int, product: str, machine: Optional[int], start_date: datetime, duration: timedelta) -> str:
        new_task = Task(amount_asked, amount_asked, product, machine, start_date, duration)
        best_machine = None
        best_start_time = None
        min_end_time = datetime.max

        for i, machine_schedule in enumerate(self.schedules):
            print(i,'HOLA')
            setup_time = self.calculate_setup_time(machine_schedule, new_task)
            new_task.start_date = start_date + setup_time
            new_task.end_date = new_task.start_date + duration

            if self.can_insert(machine_schedule, new_task):
                if new_task.end_date < min_end_time:
                    best_machine = i
                    best_start_time = new_task.start_date
                    min_end_time = new_task.end_date

        if best_machine is not None:
            new_task.machine = best_machine
            new_task.start_date = best_start_time
            new_task.end_date = best_start_time + duration
            self.schedules[best_machine].append(new_task)
            self.schedules[best_machine].sort(key=lambda task: task.start_date)
            return self.get_schedule()
        else:
            return "Task cannot be inserted due to a scheduling conflict."

    def delete_task(self, amount_asked: int, product: str, machine: int) -> str:
        for task in self.schedules[machine]:
            if task.amount_asked == amount_asked and task.product == product:
                self.schedules[machine].remove(task)
                return self.get_schedule()
        return "Task not found."

    def automated_scheduling(self, tasks: List[Task]) -> str:
        tasks.sort(key=lambda task: task.start_date)
        self.schedules = [[] for _ in range(self.num_machines)]
        for task in tasks:
            result = self.insert_task(task.amount_asked, task.product, None, task.start_date, task.duration)
            if "Task cannot be inserted" in result:
                return "Automated scheduling failed due to a conflict."
        return self.get_schedule()

    def get_schedule(self) -> str:
        schedule_output = []
        for i, machine_schedule in enumerate(self.schedules):
            schedule_output.append(f"Machine {i}:")
            for task in machine_schedule:
                schedule_output.append(repr(task))
        return '\n'.join(schedule_output)

# Example usage
if __name__ == "__main__":
    setup_times = {
        ("GlassA", "GlassB"): timedelta(minutes=30),
        ("GlassB", "GlassA"): timedelta(minutes=20),
        ("GlassA", "GlassC"): timedelta(minutes=25),
        ("GlassC", "GlassA"): timedelta(minutes=25),

    }

    gm = GlassMaker(num_machines=2, setup_times=setup_times)
    print(gm.insert_task(10, "GlassA", None, datetime(2023, 6, 1, 9, 0), timedelta(hours=2)))
    print(gm.insert_task(5, "GlassB", None, datetime(2023, 6, 1, 11, 0), timedelta(hours=1)))
    print(gm.delete_task(10, "GlassA", 0))

    tasks = [
        Task(10, 10, "GlassC", None, datetime(2023, 6, 1, 8, 0), timedelta(hours=2)),
        Task(5, 5, "GlassD", None, datetime(2023, 6, 1, 10, 0), timedelta(hours=1)),
        Task(7, 7, "GlassE", None, datetime(2023, 6, 1, 12, 0), timedelta(hours=1.5)),
    ]
    print(gm.automated_scheduling(tasks))


0 HOLA
1 HOLA
Machine 0:
Task(Product: GlassA, Amount Asked: 10, Amount Able: 10, Machine: 0, Start: 2023-06-01 09:00:00, End: 2023-06-01 11:00:00)
Machine 1:
0 HOLA
1 HOLA
Machine 0:
Task(Product: GlassA, Amount Asked: 10, Amount Able: 10, Machine: 0, Start: 2023-06-01 09:00:00, End: 2023-06-01 11:00:00)
Machine 1:
Task(Product: GlassB, Amount Asked: 5, Amount Able: 5, Machine: 1, Start: 2023-06-01 11:00:00, End: 2023-06-01 12:00:00)
Machine 0:
Machine 1:
Task(Product: GlassB, Amount Asked: 5, Amount Able: 5, Machine: 1, Start: 2023-06-01 11:00:00, End: 2023-06-01 12:00:00)
0 HOLA
1 HOLA
0 HOLA
1 HOLA
0 HOLA
1 HOLA
Machine 0:
Task(Product: GlassC, Amount Asked: 10, Amount Able: 10, Machine: 0, Start: 2023-06-01 08:00:00, End: 2023-06-01 10:00:00)
Task(Product: GlassD, Amount Asked: 5, Amount Able: 5, Machine: 0, Start: 2023-06-01 10:00:00, End: 2023-06-01 11:00:00)
Task(Product: GlassE, Amount Asked: 7, Amount Able: 7, Machine: 0, Start: 2023-06-01 12:00:00, End: 2023-06-01 13:30:00)
